In [1]:
import sklearn
sklearn.__version__
from sklearn.datasets import fetch_20newsgroups

In [2]:
dataset = fetch_20newsgroups()

In [3]:
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [28]:
raw_text = dataset.data
target = dataset.target
print(raw_text[500],dataset.target_names[target[500]])

From: bjorndahl@augustana.ab.ca
Subject: Re: document of .RTF
Organization: Augustana University College, Camrose, Alberta
Lines: 10

In article <1993Mar30.113436.7339@worak.kaist.ac.kr>, tjyu@eve.kaist.ac.kr (Yu TaiJung) writes:
> Does anybody have document of .RTF file or know where I can get it?
> 
> Thanks in advance. :)

I got one from Microsoft tech support.

-- 
Sterling G. Bjorndahl, bjorndahl@Augustana.AB.CA or bjorndahl@camrose.uucp
Augustana University College, Camrose, Alberta, Canada      (403) 679-1100
 comp.os.ms-windows.misc


## Preprocess text

0. split metadata and text. Pick only Subject
1. Lowercase words
2. delete email address 
3. Delete numbers, punctuations: \[.,-+><&/*()|"':;?{}[]`~\]
4. Replace \$ sign by a 
4. Delete human names

In [ ]:
import re
# split metadata and text
def split_metadata(data):
    occurrence = text.find("\n\n")
    metadata = text[0:occurrence]
    text = text[occurrence+2]
    return metadata, text
# get subject from metadata
def get_subject(metadata):
    # subject start with Subject: end with \n
    regex = 'Subject: (.*)\n'
    match = re.match(regex, metadata)
    return match.group(1)
# delete email address
def del_email(text):
    regex = ' .*@.* '
    return re.sub(regex, "", text)
# remove special characters
def remove(text):
    regex = ''